In [1]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time
import pandas
import traceback

import matplotlib.pyplot as plt
import numpy as np
import cv2
from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection, Point, MultiPoint
from shapely.affinity import translate

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.utils import *
from LazyLuna.Figures import *
from LazyLuna.Views import *

In [2]:
# basepaths
bp        = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil'
bp_annos  = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/NI2'
bp_cases  = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/Cases'
bp_imgs   = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/Imgs'

In [ ]:
parse_cvi42ws(bp_annos, bp_annos, process=True, debug=True)

In [3]:
lax_cine_view = LAX_CINE_View()
cases = []

# get the paths of imgs and annotations via SeriesInstanceUIDs
paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)
paths = [p for p in paths if '20220208' in p[0]]
print(paths)
for imgp, annop in paths:
    #if not '1.3.12.2.1107.5.2.43.167071.30000021082405410237400000007' in annop: continue
    try:
        print(imgp, annop)
        st = time()
        case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
        case = lax_cine_view.initialize_case(case)
        case.store(bp_cases)
        cases.append(case)
        print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)
    except Exception as e:
        print('Failed at: ', imgp, annop, ' , Exception: ', e)
        print(traceback.format_exc())
    print()

[('/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/Imgs/NICK_20220208_162522', '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/NI2/1.3.6.1.4.1.53684.1.1.2.3861684289.2996.1645800691.4257')]
/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/Imgs/NICK_20220208_162522 /Users/dietrichhadler/Desktop/Daten/Atria_Hadil/NI2/1.3.6.1.4.1.53684.1.1.2.3861684289.2996.1645800691.4257
{(0, 20): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800696.4980', (0, 6): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4784', (0, 12): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800691.4354', (0, 11): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800695.4884', (0, 25): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4662', (0, 10): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4759', (0, 17): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800691.4299', (0, 18): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800691.4314', (0, 8): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800692.4419', (0, 0): '1.3.6.1

{(0, 20): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800696.4980', (0, 6): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4784', (0, 12): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800691.4354', (0, 11): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800695.4884', (0, 25): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4662', (0, 10): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4759', (0, 17): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800691.4299', (0, 18): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800691.4314', (0, 8): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800692.4419', (0, 0): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800692.4377', (0, 3): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4692', (0, 14): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800695.4817', (0, 19): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800695.4942', (0, 2): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.1645800694.4714', (0, 16): '1.3.6.1.4.1.53684.1.1.4.3861684289.2996.16

In [6]:
cases = [pickle.load(open(os.path.join(bp_cases, p), 'rb')) for p in os.listdir(bp_cases)]
view  = LAX_CINE_View()
cases = [view.customize_case(c) for c in cases if c.reader_name=='NI2']
print(len(cases))

1


In [14]:
case = cases[0]
cats = case.categories
#print([c.name for c in cats if 'LAX 4CV' in c.name])
cat = [c for c in cats if 'LAX 4CV LAES' in c.name][0]
print(cat)
print(cat.phase)
for p in range(cat.nr_phases):
    print(cat.get_anno(0,p).anno.keys())
    if len(cat.get_anno(0,p).anno.keys())==0: continue
    img  = cat.get_img(0,p)
    anno = cat.get_anno(0,p)
    fig, ax = plt.subplots(1,1,figsize=(5,5))
    ax.imshow(img, cmap='gray')
    anno.plot_all_contour_outlines(ax)
    plt.show()
    print()
    #break

nan
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])
dict_keys([])


In [ ]:
from tensorflow.keras.utils import Sequence
import imgaug
from imgaug.augmentables.batches import UnnormalizedBatch
from imgaug import augmenters as iaa

import math

# custom z score augmenter
def func_images(images, random_state, parents, hooks):
    return [(image-np.mean(image)) / np.std(image) for image in images]
def func_polygons(polygons_on_images, random_state, parents, hooks):
    return polygons_on_images
custom_zscore_augmenter = iaa.Lambda(func_images=func_images, func_polygons=func_polygons)

# order is important 
aug = iaa.Sequential([
    iaa.PadToFixedSize(width=360, height=360, position='center', pad_mode=["linear_ramp"]), #constant, edge, linear_ramp, maximum, median, minimum, reflect, symmetric, wrap
    iaa.Sometimes(0.6, iaa.Affine(rotate = (-180, 180), scale = {"x": (0.8, 1.2), "y": (0.8, 1.2)})),      # rotation and scaling performed quite often
    iaa.Fliplr(0.5),
    iaa.Sometimes(0.6, iaa.Affine(translate_percent={"x": (-0.07, 0.07), "y": (-0.07, 0.07)}, shear=(-5, 5))), # translation and shearing occurs, but not as often
    iaa.CropToFixedSize(width=192, height=192, position='center'),
    custom_zscore_augmenter,                                                  # a custom zscore is applied (see above) after cropping, otherwise the means/variances are distorted significantly
    iaa.Sometimes(0.15, iaa.AveragePooling(((1, 2), (1, 2)))),                # average pooling mimics a downsampling of the x or y axis
    iaa.Sometimes(0.25, iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.07))), # some noise, 0.2 definitely too high, 0.05 not visible, 0.07 looks good
    iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 1.5))),                    # weak blur and occasionally (1.5% of cases) strong blur
    iaa.Sometimes(0.25, iaa.Multiply((0.7, 1.3))),                             # brightness 
    iaa.Sometimes(0.25, iaa.LinearContrast((0.7, 1.4))),                      # contrast augmentation
], random_order=False)                                                        # apply augmenters in random order

# order is important 
no_aug = iaa.Sequential([
    iaa.PadToFixedSize( width=360, height=360, position='center', pad_mode=["linear_ramp"]), #constant, edge, linear_ramp, maximum, median, minimum, reflect, symmetric, wrap
    iaa.CropToFixedSize(width=192, height=192, position='center'),
    custom_zscore_augmenter,                                                  # a custom zscore is applied (see above) after cropping, 
], random_order=False)                                                        # apply augmenters in random order

In [ ]:
class CatchGenerator(Sequence):
    def __init__(self, basepath, epochs_max, batch_size, k, k_max, provide_validation_set, aug, no_aug):
        self.basepath = basepath
        case_paths = [p for p in os.listdir(self.basepath) if p.endswith('.pickle')]
        self.cases = [pickle.load(open(os.path.join(basepath, fname), 'rb')) for fname in case_paths]
        self.cases = [c for c in self.cases if c.reader_name=='Gold']
        print('Nr of cases: ', len(self.cases))
        lax_cine_view = LAX_CINE_View()
        self.cases = [lax_cine_view.customize_case(case) for case in self.cases]
        self.tr_cases, self.val_cases = self.get_kcross_datasets(self.cases, k, k_max)
        self.epochs_max = epochs_max
        self.batch_size = batch_size
        self.aug        = aug
        self.no_aug     = no_aug
        self.k, self.k_max          = k, k_max
        self.provide_validation_set = provide_validation_set
        self.gen = self._multiprocessing_augmented_batches()
        
    def __len__(self):
        cases = self.val_cases if self.provide_validation_set else self.tr_cases
        return math.ceil(sum(2 for case in cases) / self.batch_size)
        
    def get_kcross_datasets(self, cases, k, k_max):
        n           = len(cases)
        k_nr        = n // k_max
        # HACKED FOR IDEATION STAGE ### TODO, correct ###
        train_cases = cases
        val_cases   = cases
        return train_cases, val_cases
    
    def _unaugmented_batch(self):
        scale = 1.0
        cases = self.val_cases if self.provide_validation_set else self.tr_cases
        imgs, polys = [], []
        for i in range(self.batch_size):
            case    = cases[np.random.randint(0, len(cases))]
            cats    = [c for c in case.categories if 'LAX 4CV' in c.name]
            cat     = cats[np.random.randint(0, len(cats))]
            d       = np.random.randint(0, cat.nr_slices)
            p       = cat.get_phase()
            img     = cat.get_img(d, p).astype(np.float32)
            h, w    = img.shape
            #img     = cv2.resize(img, (int(np.round(w*scale)), int(np.round(h*scale))), interpolation=cv2.INTER_CUBIC)
            anno    = cat.get_anno(d, p)
            lv_ext  = anno.get_point('lv_lax_extent')     if anno.has_point('lv_lax_extent')     else None
            ra_ext  = anno.get_point('laxRaExtentPoints') if anno.has_point('laxRaExtentPoints') else None
            la_ext  = anno.get_point('laxLaExtentPoints') if anno.has_point('laxLaExtentPoints') else None
            
            if anno.has_point('lv_lax_extent'): 
                lv_ext1 = lv_ext[0].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                lv_ext2 = lv_ext[1].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                lv_ext3 = lv_ext[2].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                lv_ext1 = imgaug.augmentables.polys.Polygon(np.array(lv_ext1), 'lv_ext1')
                lv_ext2 = imgaug.augmentables.polys.Polygon(np.array(lv_ext2), 'lv_ext2')
                lv_ext3 = imgaug.augmentables.polys.Polygon(np.array(lv_ext3), 'lv_ext3')
            
            if anno.has_point('laxRaExtentPoints'):
                ra_ext1 = ra_ext[0].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                ra_ext2 = ra_ext[1].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                ra_ext3 = ra_ext[2].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                ra_ext1 = imgaug.augmentables.polys.Polygon(np.array(ra_ext1), 'ra_ext1')
                ra_ext2 = imgaug.augmentables.polys.Polygon(np.array(ra_ext2), 'ra_ext2')
                ra_ext3 = imgaug.augmentables.polys.Polygon(np.array(ra_ext3), 'ra_ext3')
            
            if anno.has_point('laxLaExtentPoints'):
                la_ext1 = la_ext[0].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                la_ext2 = la_ext[1].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                la_ext3 = la_ext[2].buffer(4.0).simplify(0.01, preserve_topology=True).exterior.coords
                la_ext1 = imgaug.augmentables.polys.Polygon(np.array(la_ext1), 'la_ext1')
                la_ext2 = imgaug.augmentables.polys.Polygon(np.array(la_ext2), 'la_ext2')
                la_ext3 = imgaug.augmentables.polys.Polygon(np.array(la_ext3), 'la_ext3')
            
            poly    = [c for c in [lv_ext1, lv_ext2, lv_ext3, ra_ext1, ra_ext2, ra_ext3, la_ext1, la_ext2, la_ext3] if c]
            imgs.append(img); polys.append(poly)
        return imgs, polys
    
    def _make_unaugmented_batch(self, imgs, polys):
        batch = UnnormalizedBatch(images=imgs, polygons=polys)
        return batch
    
    def _unaugmented_batch_generator(self):
        for i in range(len(self)+1):
            imgs, polys = self._unaugmented_batch()
            batch = self._make_unaugmented_batch(imgs, polys)
            yield batch
            
    def _multiprocessing_augmented_batches(self):
        unaug  = self._unaugmented_batch_generator()
        for i, batch in enumerate(unaug):
            imgs1  = batch.images_unaug
            polys1 = batch.polygons_unaug
            batches_aug = list(self.aug.augment_batches(batch, background=False))  # list() converts generator to list
            imgs        = batches_aug[0].images_aug
            polys       = batches_aug[0].polygons_aug
            img_batch, mask_batch = [], []
            for j in range(batch_size):
                img, poly = imgs[j], polys[j]
                h, w = img.shape
                mask = np.zeros((h, w, 9))
                for k in range(len(poly)):
                    mask[:,:,k] = utils.to_mask(poly[k].to_shapely_polygon(), h, w)
                    #mask[:,:,k] = cv2.GaussianBlur(mask[:,:,k],(5,5),cv2.BORDER_CONSTANT)
                    mask[:,:,k] = cv2.GaussianBlur(mask[:,:,k],(9,9),cv2.BORDER_CONSTANT)
                img_batch .append(img)
                mask_batch.append(mask)
            yield img_batch, mask_batch
    
    def __getitem__(self, index):
        img_batch, mask_batch = next(self.gen)
        return (np.asarray(img_batch), np.asarray(mask_batch))
        
    def on_epoch_end(self):
        self.gen = self._multiprocessing_augmented_batches()        

In [ ]:
lax_cine_view = LAX_CINE_View()
for case in cases:
    print(case.categories)
    print(case.case_name)
    print('Clinical Results')
    case = lax_cine_view.customize_case(case)
    #for cr in case.crs:
    #    print(cr.name)
    #    print(cr.name, cr.get_cr(string=True), cr.unit)

    cats = ['LAX 4CV LVES', 'LAX 4CV LVED']
    for cat in case.categories:
        if cat.name in cats:
            print(cat.name)
            #for p in range(cat.nr_phases):
            fig, ax = plt.subplots(1,1,figsize=(9,9))
            img  = cat.get_img (0, cat.phase)
            anno = cat.get_anno(0, cat.phase)
            ax.imshow(img, cmap='gray')#; ax.axis('off')
            anno.plot_all_contour_outlines(ax)
            plt.show()
    print()

In [ ]:
img_h, img_w, img_c, out_c = 128, 128, 1, 5
d, st_ch, dr = 5, 16, 0.1

#bp = '/media/omega/Daten1/CATCH/CS/Cases'
bp  = '/Users/dietrichhadler/Desktop/Daten/Fat/Cases'

epochs_max = 3
batch_size = 16
k = 0
k_max = 2
provide_validation_set = False

gen = CatchGenerator(bp_cases, epochs_max, batch_size, k, k_max, 
                     provide_validation_set, aug, no_aug)

In [ ]:
for _ in range(10):
    img_batch, poly_batch = gen._unaugmented_batch()
    img_batch, poly_batch = aug(images=img_batch, polygons=poly_batch)

    for img, poly in zip(img_batch, poly_batch):
        h, w = img.shape
        mask = np.zeros((h, w, 9))
        for k in range(len(poly)):
            mask[:,:,k] = utils.to_mask(poly[k].to_shapely_polygon(), h, w)
            mask[:,:,k] = cv2.GaussianBlur(mask[:,:,k],(9,9),cv2.BORDER_CONSTANT)
            mask[:,:,k] = mask[:,:,k] / np.max(mask[:,:,k])
        fig, axes = plt.subplots(1,3,figsize=(9,23))
        for ax in axes: ax.axis('off')
        axes[0].imshow(img, cmap='gray')
        axes[1].imshow(img + np.sum(mask, axis=-1)*5, cmap='gray')
        axes[2].imshow(np.sum(mask, axis=-1), cmap='gray')
        fig.tight_layout()
        plt.show()

In [ ]:
for img_batch, mask_batch in gen._multiprocessing_augmented_batches():
    for i in range(len(img_batch)):
        img, mask = img_batch[i], mask_batch[i]
        fig, ax = plt.subplots(1,1,figsize=(6,6))
        ax.imshow(img + np.sum(mask, axis=-1)*1.5, cmap='gray')
        plt.show()

In [ ]:
for _ in range(1):
    import time
    unaug  = gen._unaugmented_batch_generator()
    batch  = next(unaug)
    imgs1  = batch.images_unaug
    polys1 = batch.polygons_unaug

    time_start = time.time()
    #batches_aug = list(aug.augment_batches(batch, background=False))  # list() converts generator to list
    print(dir(batch))
    batches_aug = batch
    time_end = time.time()
    print("Augmentation done in %.2fs" % (time_end - time_start,))

    #imgs  = batches_aug[0].images_aug
    #polys = batches_aug[0].polygons_aug
    imgs  = batches_aug.images_unaug
    polys = batches_aug.polygons_unaug

    for i in range(batch_size):
        img, poly = imgs[i], polys[i]
        fig, ax = plt.subplots(1,1)
        ax.imshow(img); ax.axis('off')
        print(len(poly))
        print([p.label for p in poly])
        for p in poly:
            print(len(p.exterior))
            p   = np.concatenate((p.exterior, [p.exterior[0]]))
            x,y = zip(*p)
            ax.plot(x,y)
        plt.show()


In [ ]:
for _ in range(1):
    import time
    unaug  = gen._unaugmented_batch_generator()
    batch  = next(unaug)
    imgs1  = batch.images_unaug
    polys1 = batch.polygons_unaug

    time_start = time.time()
    batches_aug = list(aug.augment_batches(batch, background=False))  # list() converts generator to list
    time_end = time.time()
    print("Augmentation done in %.2fs" % (time_end - time_start,))

    imgs  = batches_aug[0].images_aug
    polys = batches_aug[0].polygons_aug

    for i in range(batch_size):
        img, poly = imgs[i], polys[i]
        h, w = img.shape
        mask = np.zeros((h, w, 5))
        for j in range(len(poly)): mask[:,:,j] = to_mask(poly[j].to_shapely_polygon(), h, w)
            
        fig, axes = plt.subplots(1,2, figsize=(25,15))
        for j in range(2): axes[j].imshow(img, cmap='gray'); axes[j].axis('off')
        
        #['lv_lax_endo', 'lv_lax_epi', 'rv_lax_endo', 'ra', 'la']
        mask_sum = np.sum(mask, axis=-1)
        axes[1].imshow(img + mask_sum*5, cmap='gray')
        #axes[1].imshow(img + mask[:,:,0]*3 + mask[:,:,1]*3, cmap='gray')
        plt.show()

In [ ]:
for img_batch, mask_batch in gen._multiprocessing_augmented_batches():
    for i in range(len(img_batch)):
        img, mask = img_batch[i], mask_batch[i]
        print(np.mean(img), np.mean(mask))
        plt.imshow(img + 5*np.sum(mask, axis=-1), cmap='gray')
        plt.show()

In [ ]:
import imgaug as ia
from imgaug.augmentables.kps import Keypoint, KeypointsOnImage

for case in cases:
    print(case.case_name, case.reader_name)
    cat1  = [c for c in case.categories if c.name=='LAX 4CV LVES'][0]
    cat2  = [c for c in case.categories if c.name=='LAX 4CV LVED'][0]
    for cat in [cat1, cat2]:
        img   = cat.get_img (0, cat.phase).astype(np.float32)
        h, w  = img.shape
        anno  = cat.get_anno(0, cat.phase)
        ps1   = list(anno.get_point('lv_lax_extent').geoms)     if anno.has_point('lv_lax_extent')     else None
        ps2   = list(anno.get_point('laxRaExtentPoints').geoms) if anno.has_point('laxRaExtentPoints') else None
        ps3   = list(anno.get_point('laxLaExtentPoints').geoms) if anno.has_point('laxLaExtentPoints') else None
        print(ps1); print(ps2); print(ps3)
        ps = []
        if ps1 is not None: ps += ps1
        if ps2 is not None: ps += ps2
        if ps3 is not None: ps += ps3
        #try:                   ps = ps1 + ps2 + ps3
        #except Exception as e: print(e)

        kps = [Keypoint(x=int(p.x), y=int(p.y)) for p in ps]

        fig, axes = plt.subplots(1,2,figsize=(10,6))
        axes[0].imshow(img, cmap='gray')
        xs, ys = [point.x for point in ps], [point.y for point in ps]
        axes[0].scatter(xs, ys, c='r',s=15,marker='x')
        #plt.show()
        
        kpsoi = KeypointsOnImage(kps, shape=img.shape)
        print('First keypoints:')
        print(kpsoi)
        
        img_aug, kpsoi_aug = aug(image=img, keypoints=kpsoi)
        ps_aug = [(kp.x,kp.y) for kp in kpsoi_aug]
        #ia.imshow(kpsoi_aug.draw_on_image(img_aug, color=4, size=3))

        #print(kpsoi_aug[0].x, kpsoi_aug[0].y)
        ps_aug = [Point(p.x,p.y) for p in kpsoi_aug]
        #print('Augmented keypoints:')
        #print(kpsoi_aug)
        #for p in ps_aug: print(p.x, p.y)
        axes[1].imshow(img_aug, cmap='gray')
        xs, ys = [point.x for point in ps_aug], [point.y for point in ps_aug]
        axes[1].scatter(xs, ys, c='r',s=15,marker='x')
        plt.show()